In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import os

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import scipy.stats as stats

import seaborn as sns

from datetime import datetime

import re

from tqdm.notebook import tqdm

import dask.dataframe as dd
from dask.distributed import Client

sys.path.insert(0, 'tools')

from tools import * 

/tmp/ipykernel_5316/962277102.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 20,Total memory: 31.03 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44225,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 31.03 GiB
Comm: tcp://127.0.0.1:39153,Total threads: 4
Dashboard: http://127.0.0.1:35141/status,Memory: 6.21 GiB
Nanny: tcp://127.0.0.1:38657,


In [5]:
config = pd.Series({
    'path': os.path.abspath('dades'),
    'file_type':'csv',
    'years':[2019],
    'dataset': 'BicingNou_INFORMACIO'
})


In [6]:
%%time

data = read_dask_dataframes(config.path, config.file_type, config.dataset, config.years)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

CPU times: user 145 ms, sys: 33 ms, total: 178 ms
Wall time: 172 ms


In [7]:
config_year = pd.Series({
    'path':config.path,
    'file_type':config.file_type,
    'year':2019,
    'dataset': 'BicingNou_ESTACIONS_MOD'
})

In [17]:
data[2019] = data[2019].drop(columns=['physical_configuration', 'last_updated', 'ttl'], axis=1)

ValueError: Metadata inference failed in `drop_by_shallow_copy`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
KeyError('[None] not found in axis')

Traceback:
---------
  File "/home/homeros/miniconda3/envs/rcnn/lib/python3.8/site-packages/dask/dataframe/utils.py", line 193, in raise_on_meta_error
    yield
  File "/home/homeros/miniconda3/envs/rcnn/lib/python3.8/site-packages/dask/dataframe/core.py", line 6810, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/home/homeros/miniconda3/envs/rcnn/lib/python3.8/site-packages/dask/dataframe/utils.py", line 766, in drop_by_shallow_copy
    df2.drop(columns=columns, inplace=True, errors=errors)
  File "/home/homeros/miniconda3/envs/rcnn/lib/python3.8/site